# 0.0. Imports

In [1]:
## biblio que salva objetos criados para serem utilizados posteriormente
import joblib
## biblio para manipular dados tabulares
import pandas as pd
### biblio para validação dos dados de entrada
import pandera  
from pandera import Check, Column, DataFrameSchema
### biblio para separação de amostras de treino e teste
from sklearn.model_selection import train_test_split
### método que auxilia no processo do fluxo de tratamento dos dados de treino e teste sem enviesar os dados
from sklearn.pipeline import Pipeline
### métodos de preprocessamento
from feature_engine.discretisation import EqualFrequencyDiscretiser, EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import RobustScaler, StandardScaler
## método para avaliar performance do modelo  
from sklearn.metrics import roc_auc_score
## método que disponibiliza o modelo utilizado 
from sklearn.linear_model import LogisticRegression

C:\Users\alanc\AppData\Local\Temp\ipykernel_12716\1383000715.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 0.1. Utils
Funções e variáveis reutilizáveis

In [2]:
columns_to_use = ['target', 'TaxaDeUtilizacaoDeLinhasNaoGarantidas',
       'Idade', 'NumeroDeVezes30-59DiasAtrasoNaoPior', 'TaxaDeEndividamento',
       'RendaMensal', 'NumeroDeLinhasDeCreditoEEmprestimosAbertos',
       'NumeroDeVezes90DiasAtraso', 'NumeroDeEmprestimosOuLinhasImobiliarias',
       'NumeroDeVezes60-89DiasAtrasoNaoPior', 'NumeroDeDependentes']

# 1.0. Data load

In [3]:
### classe que encapsula etapa de carregamento dos dados
class DataLoad:
    """Class data load"""
    
    def __init__(self) -> None:
        pass 
    
    def load_data(self) -> pd.DataFrame:
        """Funcao vai carregar os dados
        
        return:
            pandas DataFrame"""
            
        loaded_data = pd.read_csv('../data/raw/train.csv')
        return loaded_data

In [4]:
### Instanciando a classe DataLoad
dl = DataLoad()

### Instanciando objeto que receberá dataframe treino
df = dl.load_data()[columns_to_use]
df.head()

,target,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


# 2.0. Data validation

In [5]:
class DataValidation:
    """Classe usada para checar/validar a conformidade do dataframe de treino na entrada"""
    def __init__(self, columns_to_use) -> None: ## initialize método recebe as colunas que devem estar contidas nos dados de treino
        self.columns_to_use = columns_to_use
    
    def check_shape_data(self, dataframe: pd.DataFrame) -> bool:  ## recebe um dataframe e verifica se contém as colunas desejadas
        try:
            print('Validacao iniciou')
            dataframe.columns = self.columns_to_use
            return True 
        except Exception as e:
            print(f'Validacao errou: {e}')
            return False
        
    ## método que checa coluna por coluna relativo aos tipos de dados e Check(lambda x: x > 0), coerce=True verifica nulos para target
        ### para outras colunas aceita colunas com nulos pois posteriormente será tratada.
    def check_columns(self, dataframe: pd.DataFrame) -> bool: 
        schema = DataFrameSchema(
                {
                    "target": Column(int, Check.isin([0, 1]), Check(lambda x: x > 0), coerce=True),
                    "TaxaDeUtilizacaoDeLinhasNaoGarantidas": Column(float, nullable=True),
                    "Idade": Column(int, nullable=True),
                    "NumeroDeVezes30-59DiasAtrasoNaoPior": Column(int, nullable=True),
                    "TaxaDeEndividamento": Column(float, nullable=True),
                    "RendaMensal": Column(float, nullable=True),
                    "NumeroDeLinhasDeCreditoEEmprestimosAbertos": Column(int, nullable=True),
                    "NumeroDeVezes90DiasAtraso": Column(int, nullable=True),
                    "NumeroDeEmprestimosOuLinhasImobiliarias": Column(int, nullable=True),
                    "NumeroDeVezes60-89DiasAtrasoNaoPior": Column(int, nullable=True),
                    "NumeroDeDependentes": Column(float, nullable=True)
                }
            )
        try:
            schema.validate(dataframe)
            print("Validation columns passed...")
            return True
        except pandera.errors.SchemaErrors as exc:
            print("Validation columns failed...")
            pandera.display(exc.failure_cases)
        return False
    
    ## método executor da classe que irá chamar os outros métodos e orquestrar as validações
    def run(self, dataframe: pd.DataFrame) -> bool:
        if self.check_shape_data(dataframe) and self.check_columns(dataframe):
            print('Validacao com sucesso.')
            return True 
        else:
            print('Validacao falhou.')
            return False

In [6]:
## Instanciando a classe e passar os argumentos necessários
dv = DataValidation(columns_to_use)

## chamando a classe com o método principal
dv.run(df)

Validacao iniciou
Validation columns passed...
Validacao com sucesso.


True

# 3.0 Data Transformation 
Realizar algum tipo de transformação nos dados

In [7]:
class DataTransformation:
    def __init__(self, dataframe: pd.DataFrame, target_name: str):
        self.dataframe = dataframe 
        self.target_name = target_name 
        
    def train_test_spliting(self): ### função que realiza a separação dados de treino e validação
        X = self.dataframe.drop(self.target_name, axis=1)
        y = self.dataframe[self.target_name]
        
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y)
        
        return X_train, X_valid, y_train, y_valid

In [8]:
dt = DataTransformation(df, 'target')

X_train, X_valid, y_train, y_valid = dt.train_test_spliting()

In [9]:
X_train.shape

(112500, 10)

In [10]:
X_valid.shape

(37500, 10)

# 4.0 Data preprocess

Feature Engineering, data preparation (encoding, scaler) and fillna 

In [ ]:
class DataPreprocess: ### classe que recebe um dataframe e o objeto instanciado Pipeline
    def __init__(self, dataframe: pd.DataFrame,
                       pipe: Pipeline):
        self.dataframe = dataframe
        self.pipe = pipe 
        
    def pipeline(self): ### ira fitar os dados de treino
        train_pipe = self.pipe
        train_pipe.fit(self.dataframe)
        return train_pipe 
    
    def run(self):
        print('Preprocessador iniciou...')
        trained_pipeline = self.pipeline() ### traz o pipeline treinado
        data_preprocessed = trained_pipeline.transform(self.dataframe) ### realiza transformação
        return data_preprocessed

In [ ]:
### imputer é para tratar nulos , já os outros são para padronizaçâo de variáveis
pipe = Pipeline([('imputer', MeanMedianImputer(variables=['RendaMensal',
                                                          'NumeroDeDependentes'])),
                 ('discretizer', EqualFrequencyDiscretiser(variables=['TaxaDeUtilizacaoDeLinhasNaoGarantidas',
                                                                      'TaxaDeEndividamento',
                                                                      'RendaMensal'])),
                 ('scaler', SklearnTransformerWrapper(StandardScaler()))])

In [ ]:
## chama classe
dp = DataPreprocess(X_train, pipe)

In [ ]:
### sai com dataframe processado
X_train_processed = dp.run()

In [ ]:
### dados de entrada sem preprocessamento
X_train.head()

In [ ]:
### dados de entrada com preprocessamento
X_train_processed.head()

In [ ]:
### salvar a arquitetura do preprocessador utilizado
joblib.dump(dp.pipeline(),
            'preprocessor.joblib')

# 5.0. Train models

In [ ]:
class TrainModels:
    def __init__(self, dados_X: pd.DataFrame,
                       dados_y: pd.DataFrame):
        self.dados_X = dados_X 
        self.dados_y = dados_y 
        
    def train(self, model):
        model.fit(self.dados_X, self.dados_y)
        joblib.dump(model, 'modelo.joblib')
        return model 
    
    def predict(self, dados_para_prever: pd.DataFrame):
        model_fitted = self._load_model()
        dados_pred = model_fitted.predict_proba(dados_para_prever)
        return dados_pred
    
    def _load_model(self):
        model = joblib.load('modelo.joblib')
        return model

In [ ]:
tm = TrainModels(dados_X=X_train_processed,
                 dados_y = y_train)

In [ ]:
tm.train(model=LogisticRegression())

In [ ]:
y_train_pred = tm.predict(X_train_processed)

In [ ]:
y_train_pred

# 6.0. Models evaluation

In [ ]:
preprocessor = dp.pipeline()

In [ ]:
preprocessor

In [ ]:
X_valid_processed = preprocessor.transform(X_valid)

In [ ]:
y_valid_pred = tm.predict(X_valid_processed)

In [ ]:
class ModelEvaluation:
    def __init__(self):
        pass 
    
    def eval_metrics(self, dados_reais, dados_preditos):
        roc_auc = roc_auc_score(dados_reais, dados_preditos)
        return roc_auc

In [ ]:
me = ModelEvaluation()

In [ ]:
y_train_pred[:, 1]

In [ ]:
me.eval_metrics(y_train, y_train_pred[:, 1])

In [ ]:
me.eval_metrics(y_valid, y_valid_pred[:, 1])

# 7.0. Experiments

## 7.1. Experiment 1

In [ ]:
# 1. etapa
pipe = Pipeline([('imputer', MeanMedianImputer(variables=['RendaMensal',
                                                          'NumeroDeDependentes'])),
                 ('discretizer', EqualFrequencyDiscretiser(variables=['TaxaDeUtilizacaoDeLinhasNaoGarantidas',
                                                                      'TaxaDeEndividamento',
                                                                      'RendaMensal'])),
                 ('scaler', SklearnTransformerWrapper(RobustScaler()))])
dp = DataPreprocess(X_train, pipe)
X_train_processed = dp.run()

#---------------------#
#2. etapa
tm = TrainModels(dados_X=X_train_processed,
                 dados_y = y_train)
tm.train(model=LogisticRegression(penalty='l2', max_iter=1500, solver='newton-cholesky'))
y_valid_pred = tm.predict(X_valid_processed)

#---------------------#
# 3.etapa
me = ModelEvaluation()
me.eval_metrics(y_valid, y_valid_pred[:, 1])